In [ ]:
!!curl ipecho.net/plain

['35.233.222.111']

In [ ]:

import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup as bs
import re
#기존의 크롤링 방법 말고 다른 방식을 써보자

id='mugudeli601'
url = "https://www.law.go.kr/DRF/lawSearch.do?OC={}&target=prec&type=XML".format(id)
response = urlopen(url).read()
bs(response)

<?xml version="1.0" encoding="UTF-8"?><html><body><precsearch><target>prec</target>&lt;키워드&gt;*키워드&gt;<section>evtNm</section><totalcnt>84441</totalcnt><page>1</page><prec id="1">&lt;판례일련번호&gt;223803판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[손해배상(기)·위약벌]]&gt;사건명&gt;&lt;사건번호&gt;2018다248855사건번호&gt;&lt;선고일자&gt;2022.07.21선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;전원합의체 판결판결유형&gt;&lt;선고&gt;선고선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=223803&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="2">&lt;판례일련번호&gt;223801판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[토지인도]]&gt;사건명&gt;&lt;사건번호&gt;2017다236749사건번호&gt;&lt;선고일자&gt;2022.07.21선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;전원합의체 판결판결유형&gt;&lt;선고&gt;선고선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=223801&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec

In [ ]:
import re

xtree = ET.fromstring(response) #response에 저장된 string으로부터 tree를 생성한다
#first check what is in it
'''for child in xtree:
    print(':',child.tag,child.attrib)
print(xtree.find('page').text)'''
xtree[3].text #totalCnt
for node in xtree[5:]:
    print('\n\n',node.attrib)
    tags=[]
    for child in node:
        tags.append(child.tag)
    for t in tags:
        print(node.find(t).text)
totalCnt = int(xtree.find('totalCnt').text)#판례 페이지에 totalCnt라는 요소가 존재하는 것 같다. 
#왜 20개밖에 없나 했는데 원래 한 페이지당 20개라고 한다. 
#페이지마다 loop을 돌아가야 하는 것
#판례 목록 만들기 
cnt=0
page = 1
rows = []
for i in trange(int(totalCnt / 20)): #기다리는 바를 만드는 tqdm의 range 함수인 trange
    if cnt>500:
      break
    try:
        items = xtree[5:]
    except:
        break
        
    for node in items:
        data={}
        tags=[]
        for child in node:
            tags.append(child.tag)
        for t in tags:
            data[t]=re.sub(',','$',str(node.find(t).text))

        rows.append(data)
    cnt += 1
    url = "https://www.law.go.kr/DRF/lawSearch.do?OC=mugudeli601&target=prec&type=XML&page={}".format(page)
    response = urlopen(url).read()
    xtree = ET.fromstring(response)
    page+=1
#rows에 모든 정보가 담겨있음
prec_list=pd.DataFrame(rows,columns=rows[0].keys())
prec_list.head(5)
prec_list.columns
#prec_list를 traverse하며 모든 판례정보를 긁기

contents = ['사건명', '사건번호', '선고일자','판결유형','판시사항', '판결요지', '참조조문', '참조판례', '판례내용']
case_list=pd.DataFrame(columns=contents)




 {'id': '1'}
223803
손해배상(기)·위약벌
2018다248855
2022.07.21
대법원
None
민사
400101
전원합의체 판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223803&type=HTML&mobileYn=


 {'id': '2'}
223801
토지인도
2017다236749
2022.07.21
대법원
None
민사
400101
전원합의체 판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223801&type=HTML&mobileYn=


 {'id': '3'}
223807
손해배상(자)
2020다230857
2022.07.14
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223807&type=HTML&mobileYn=


 {'id': '4'}
223819
통행권존재확인등청구
2022다228544
2022.07.14
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223819&type=HTML&mobileYn=


 {'id': '5'}
223825
해임처분취소
2022두33323
2022.07.14
대법원
None
일반행정
400107
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223825&type=HTML&mobileYn=


 {'id': '6'}
223835
자동차관리법위반
2021도16578
2022.07.14
대법원
None
형사
400102
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=223835&type=HTML&mobileYn=


 {'id': '7'}
227275
손해배상(기) [과밀수용을 원인으로 한 국가배상청구 사건]
2017다

 47%|████▋     | 2001/4222 [29:53<33:10,  1.12it/s]


In [ ]:

def remove_tag(content):
    cleaned=re.sub(',','$',re.sub('<.*?>', '', content))
    return cleaned

for pid,l in zip(prec_list['판례일련번호'],prec_list['판례상세링크']):
    url="https://www.law.go.kr"+l.replace('HTML','XML')
    response=urlopen(url).read()
    xtree=ET.fromstring(response)
    row={}
    row['판례일련번호']=pid
    for content in contents:
        
        try:
            text=xtree.find(content).text
            text=remove_tag(text)
        except:
            text="-"
        row[content]=text
    case_list=pd.concat([case_list,pd.DataFrame([row],columns=row.keys())],axis=0)
case_list.reset_index(inplace=True)
        
        

        
case_list.head(5)
from pathlib import Path  
filepath = Path('/content/Level1_w_data/case_list.csv')  
case_list.to_csv(filepath,index=False)
case_list.head(10)
print(case_list['참조판례'])
type(case_list)
nonan_assistive=case_list[~(case_list['참조판례']=='-')].copy()
nonan_assistive[nonan_assistive['참조판례'].str.contains('\[[0-9]\]')]

URLError: ignored

In [ ]:
!git clone https://github.com/Mubbit/Level1_w_data.git


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.5 MB/s 
     |████████████████████████████████| 453 kB 46.5 MB/s 


In [ ]:
!python Level1_w_data/data_preprocess_doc2vec.py

::::::::::df set 120
:::::::::::re done 120
100% 120/120 [05:39<00:00,  2.83s/it]
::::::::::::tokens per doc length 120
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
Figure(640x480)


textrank

In [ ]:
! git clone https://github.com/lovit/textrank.git

In [ ]:
from data_preprocess_doc2vec import PreProcess
import pandas as pd

caselist_path=''
caselist_pd=pd.read_csv(caselist_path)
p=PreProcess(caselist)
p.set()
docs=p.df

In [ ]:
def custom_tokenizer(fn,x): 
  tokens,stopwords=fn.run_inference(x)
  return tokens

In [ ]:
from textrank import KeywordSummarizer

keyword_extractor=KeywordSummarizer(
    tokenize=lambda x:custom_tokenizer(p.run_inference,x),
    window=-1,
    verbose=False
)
keywords=keyword_extractor.summarize(docs,topk=30)
for index,word,rank in enumerate(keywords):
  print(word,rank)
  if index>10:
    break